## ДЗ по поиску

Привет! Вам надо реализивать поисковик на базе вопросов-ответов с сайта [pravoved.ru](https://pravoved.ru/questions-archive/).        
Поиск должен работать на трех технологиях:       
1. обратном индексе     
2. word2vec         
3. doc2vec      

Вы должны понять, какой метод и при каких условиях эксперимента на этом корпусе работает лучше.          
Для измерения качества поиска найдите точность (accuracy) выпадания правильного ответа на конкретный вопрос (в этой базе у каждого вопроса есть только один правильный ответ). Точность нужно измерить для всей базы.    
При этом давайте считать, что выпал правильный ответ, если он попал в **топ-5** поисковой выдачи.

> Сделайте ваш поиск максимально качественным, чтобы значение точности стремилось к 1.     
Для этого можно поэкспериментировать со следующим:       
- модель word2vec (можно брать любую из опен сорса или обучить свою)
- способ получения вектора документа через word2vec: простое среднее арифметическое или взвешивать каждый вектор в соответствии с его tf-idf      
- количество эпох у doc2vec (начинайте от 100)
- предобработка документов для обучения doc2vec (удалять / не удалять стоп-слова)
- блендинг методов поиска: соединить результаты обратного индекса и w2v, или (что проще) w2v и d2v

На это задание отведем 10 дней. Дэдлайн сдачи до полуночи 12.10.

In [1]:
import gensim
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
import numpy as np
from tqdm import tqdm
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from gensim import matutils
import string
from pymystem3 import Mystem
mystem = Mystem()
from gensim.models.fasttext import FastText
from tqdm import tqdm_notebook
from judicial_splitter import splitter
import pickle
import os
import json

In [2]:
import pickle

with open('qa_corpus.pkl', 'rb') as file:
    qa_corpus = pickle.load(file)

In [3]:
questions = []
answers = []

for item in qa_corpus:
    questions.append(item[0])
    answers.append(item[1])

Всего в корпусе 1384 пары вопрос-ответ

In [ ]:
len(qa_corpus)

Первый элемент блока это вопрос, второй - ответ на него

In [ ]:
qa_corpus[0]

In [4]:
def preprocessing(input_text, del_stopwords=True, del_digit=False):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation + '»«–…—') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]
    
    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

### w2v

In [5]:
model_w2v = Word2Vec.load('araneum_none_fasttextcbow_300_5_2018.model')

In [6]:
def get_w2v_vectors(model_w2v, lemmas):
    vec_list = []
    
    for word in lemmas:
        try:
            vec = model_w2v.wv[word]
            vec_list.append(vec)
        except:
            continue
            
    vec = sum(vec_list) / len(vec_list)
    return vec

In [7]:
def save_w2v_base(corpus):
    base = []
    for i, file in tqdm(enumerate(corpus)):
        paragraphs = splitter(file, 3)
        
        for paragraph in paragraphs:
            pr_par = preprocessing(paragraph)
            vector = get_w2v_vectors(model_w2v, pr_par)
            base.append({'id' : [i], 'text': paragraph, 'vector': vector})
    return base

In [8]:
base_ans_wv = save_w2v_base(answers)

1384it [00:22, 60.73it/s]


In [ ]:
base_ans_wv

In [9]:
base_que_wv = save_w2v_base(questions)

1384it [00:11, 119.51it/s]


### d2v

In [23]:
tagged_data = []
file = {}
i = 0

def train_doc2vec(data):
    
    for paragraph in tqdm(data):                
        tagged_data.append(TaggedDocument(words=preprocessing(paragraph), tags=[i]))    
        
    d2v_model = Doc2Vec(vector_size=100, min_count=5, alpha=0.025, min_alpha=0.025, epochs=100, workers=4, dm=1, seed=42)
    d2v_model.build_vocab(tagged_data)    
    d2v_model.train(tagged_data, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs, report_delay=60)
    
    return d2v_model

In [24]:
d2v_model = train_doc2vec(answers)

100%|██████████| 1384/1384 [00:21<00:00, 65.40it/s]


In [25]:
fname = get_tmpfile("doc2vec_model_answers")
d2v_model.save(fname)

In [26]:
def get_d2v_vectors(lemmas):
    vec = d2v_model.infer_vector(lemmas)
    return vec

In [27]:
def save_d2v_base(corpus):
    base = []
    
    for i, text in tqdm(enumerate(corpus)):
        paragraphs = splitter(text, 3)
        
        for paragraph in paragraphs:
            pr_par = preprocessing(paragraph)
            vector = get_d2v_vectors(pr_par)
            base.append({'id': [i], 'text': paragraph, 'vector': vector})
    return base

In [28]:
base_ans_dv = save_d2v_base(answers)

1384it [00:38, 35.90it/s]


In [ ]:
base_que_dv = save_d2v_base(questions)

### search

In [10]:
from gensim import matutils
import numpy as np 

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

In [17]:
def search_w2v(request, model_w2v, base_ans_wv, n_results):
    vec = get_w2v_vectors(model_w2v, request)
    similarity_dict = {}
    
    for elem in base_ans_wv:
        sim = similarity(vec, elem['vector'])
        similarity_dict[sim] = elem['text']
        
    result = [similarity_dict[sim] for sim in sorted(similarity_dict, reverse=True)[:n_results]]
    return result

In [20]:
def search(request, search_method, n_results=5, return_answer_text=False):
    
    request = preprocessing(request, del_stopwords=False)
    if search_method == 'word2vec':
        search_result = search_w2v(request, model_w2v, base_ans_wv, n_results)
    elif search_method == 'doc2vec':
        search_result = search_d2v(request, d2v_model, base_ans_dv, n_results)
    else:
        raise TypeError('unsupported search method')
    
    if not return_answer_text:
        return search_result
    
    results = [(index, answers[index]) for index in search_result]
    return results

In [30]:
accuracy_score = 0
answers_index = []
    
for i, question in enumerate(tqdm_notebook(questions)):
    
    search_result = search(question, 'word2vec')
        
    if i in search_result:
        accuracy_score += 1
        answers_index.append(i)
            
final_accuracy = accuracy_score * 100 / len(questions) 

HBox(children=(IntProgress(value=0, max=1384), HTML(value='')))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [31]:
final_accuracy

0.0